In [1]:
import pandas as pd
import numpy as np
import copy
import json
from ydata_profiling import ProfileReport

In [2]:
df = pd.read_csv('../data/credit_score/credit_score_test.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      44985 non-null  object 
 4   Age                       50000 non-null  object 
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  object 
 8   Monthly_Inhand_Salary     42502 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  int64  
 12  Num_of_Loan               50000 non-null  object 
 13  Type_of_Loan              44296 non-null  object 
 14  Delay_

In [4]:
# Checking the data types
print(df.dtypes)

# Checking for missing values
print(df.isnull().sum())

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
dtype: object
ID            

In [5]:
# List of columns that should be numeric
convert_cols = ["Age", "Annual_Income", "Outstanding_Debt", "Amount_invested_monthly", "Monthly_Balance", "Num_of_Loan", "Num_of_Delayed_Payment", "Changed_Credit_Limit"]

# Convert to numeric and force errors to NaN
for col in convert_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [6]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.642682,Low_spent_Small_value_payments,186.266702
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380,High_spent_Medium_value_payments,361.444004
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.233938,Low_spent_Medium_value_payments,264.675446
3,0x160d,CUS_0xd40,December,Aaron Maashoh,NaN,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.082511,High_spent_Medium_value_payments,343.826873
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28.0,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434


In [7]:
def convert_credit_history(value):
    if isinstance(value, str):
        years = int(value.split(" Years")[0]) if "Years" in value else 0
        months = int(value.split(" and ")[-1].split(" Months")[0]) if "Months" in value else 0
        return years + months / 12  # Convert months to decimal
    return None

df["Credit_History_Age"] = df["Credit_History_Age"].apply(convert_credit_history)


In [8]:
categorical_cols = ["Occupation", "Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour"]

for col in categorical_cols:
    df[col] = df[col].astype("category")

In [9]:
df.loc[df["Age"] < 0, "Age"] = df["Age"].median()  # Replace negative ages with median
df.loc[df["Age"] > 80, "Age"] = df["Age"].median()  # Replace unrealistic ages

df.loc[df["Num_Bank_Accounts"] < 0, "Num_Bank_Accounts"] = df["Num_Bank_Accounts"].median()  # Replace negative bank accounts with median
df.loc[df["Num_Bank_Accounts"] > 12, "Num_Bank_Accounts"] = df["Num_Bank_Accounts"].median()  # Replace unrealistically high number of bank accounts with median

df.loc[df["Num_of_Loan"] < 0, "Num_of_Loan"] = df["Num_of_Loan"].median()  # Replace negative num of loan with median
df.loc[df["Num_of_Loan"] > 12, "Num_of_Loan"] = df["Num_of_Loan"].median()  # Replace unrealistic num of loan

df.loc[df["Num_of_Delayed_Payment"] < 0, "Num_of_Delayed_Payment"] = df["Num_of_Delayed_Payment"].median()  # Replace negative num of delayed payments with median
df.loc[df["Num_of_Delayed_Payment"] > 28, "Num_of_Delayed_Payment"] = df["Num_of_Delayed_Payment"].median()  # Replace unrealistic num of delayed payments

df.loc[df["Num_Credit_Card"] < 0, "Num_Credit_Card"] = df["Num_Credit_Card"].median()  # Replace negative num of credit cards with median
df.loc[df["Num_Credit_Card"] > 12, "Num_Credit_Card"] = df["Num_Credit_Card"].median()  # Replace unrealistic num of credit cards

df.loc[df["Interest_Rate"] < 0, "Interest_Rate"] = df["Interest_Rate"].median()  # Replace negative interest rates with median
df.loc[df["Interest_Rate"] > 32, "Interest_Rate"] = df["Interest_Rate"].median()  # Replace unrealistic interest rates

df.loc[df["Num_Credit_Inquiries"] < 0, "Num_Credit_Inquiries"] = df["Num_Credit_Inquiries"].median()  # Replace negative num of credit inquiries with median
df.loc[df["Num_Credit_Inquiries"] > 50, "Num_Credit_Inquiries"] = df["Num_Credit_Inquiries"].median()  # Replace unrealistic num of credit inquiries

df.loc[df["Total_EMI_per_month"] < 0, "Total_EMI_per_month"] = df["Total_EMI_per_month"].median()  # Replace negative total EMI per month with median
df.loc[df["Total_EMI_per_month"] > 4500, "Total_EMI_per_month"] = df["Total_EMI_per_month"].median()  # Replace unrealistic total EMI per month

In [10]:
# List of columns that are numeric
numeric_cols = ["Age", "Annual_Income", "Outstanding_Debt", "Amount_invested_monthly", "Monthly_Balance", "Num_of_Loan", "Num_of_Delayed_Payment", "Num_Bank_Accounts", "Num_Credit_Card", 
                "Interest_Rate", "Num_Credit_Inquiries", "Total_EMI_per_month", "Monthly_Inhand_Salary", "Credit_History_Age"]

# Impute missing values in numeric columns
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

In [11]:
# List of numeric columns that should be integers
int_numeric_cols = ["Age", "Num_of_Loan", "Num_of_Delayed_Payment", "Num_Credit_Inquiries"]

# Convert float columns to int columns
for col in int_numeric_cols:
    df[col] = pd.to_numeric(df[col], downcast='integer', errors="coerce")

# List of remaining columns that remain floats
float_numeric_cols = ["Annual_Income", "Outstanding_Debt", "Amount_invested_monthly", "Monthly_Balance", "Num_Bank_Accounts", "Num_Credit_Card", 
                      "Interest_Rate", "Total_EMI_per_month", "Monthly_Inhand_Salary", "Credit_History_Age"]

# Round float columns to 2 digits
df[float_numeric_cols] = df[float_numeric_cols].round(2)

In [12]:
df = df.dropna(subset=["Age"])  # drop rows with no age
df = df.dropna(subset=['Changed_Credit_Limit'])  # drop rows with no changed credit limit
df = df[df["Occupation"] != "_______"]  # drop rows with not identifiable occupation
df = df[df["Credit_Mix"] != "_"]  # drop rows with not identifiable occupation
df = df[df["Payment_Behaviour"] != "!@9#%8"]  # drop rows with not identifiable occupation

In [13]:
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])  # Display only columns with missing values

Name            3388
Type_of_Loan    3914
dtype: int64


In [14]:
# For categorical columns, use mode (most frequent value):
df["Type_of_Loan"] = df["Type_of_Loan"].fillna(df["Type_of_Loan"].mode()[0])

In [16]:
# Generate the profiling report
profile = ProfileReport(df, title="YData Profiling Report CREDIT CLEANED v2", explorative=True)

# Display the report in the notebook
# profile.to_notebook_iframe()

# Alternatively, save the report to an HTML file
profile.to_file('../profiling/profiling_report_cleaned_credit_score_v2.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\svenl\miniconda3\envs\hyperRealDataDescriber\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 2 2 ... 2 2 2]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\svenl\miniconda3\envs\hyperRealDataDescriber\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 3 2 3]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
c:\Users\svenl\miniconda3\envs\hyperRealDataDescriber\Lib\site-packages\ydata_profiling\model\pandas\discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is de

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
# Once the transformations are complete, save the cleaned dataset:
df.to_csv("../data/credit_score/cleaned_credit_score_v2.csv", index=False)